In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"


In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_enums import CandleBodyType

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
import numpy as np
from numba import int64, types, typed
from numba.experimental import jitclass

spec_foo = {"data": int64[:]}


@jitclass(spec_foo)
class Foo(object):
    def __init__(self, init_data):
        self.data = init_data


# Want to have a list of Foo instances in the Bar class
# >>> Use ListType instead of List here.
# spec_bar = {"foo_list": types.ListType(Foo.class_type.instance_type)}


# @jitclass(spec_bar)
class Bar(object):
    def __init__(self, foo_list):
        # This assigns a reference to a list of Foo instances
        self.foo_list = foo_list


x = np.arange(5).astype(np.int64)
# Build a Foo instance
foo = Foo(x)

# Create a Bar instance that holds a list of Foo instance.
# >>> Pass a typed.List here.
bar = Bar(typed.List([foo]))
foo

In [4]:
bar.foo_list[0].data

array([0, 1, 2, 3, 4], dtype=int64)

In [5]:
foo.data

array([0, 1, 2, 3, 4], dtype=int64)

In [47]:
import numpy as np
from numba import int64, types, typed, optional, UnionType
from numba.experimental import jitclass

spec = {"data": int64[:]}


class EmptyClass:
    def __init__(self) -> None:
        pass

    def tester(self):
        print("hello")

    def multi_data(self):
        pass

    def print_data(self):
        pass


@jitclass(spec)
class Foo(EmptyClass):
    def __init__(self, init_data):
        self.data = init_data

    def print_data(self):
        print(self.data)


@jitclass(spec)
class Boo(EmptyClass):
    def __init__(self, init_data):
        self.data = init_data

    def multi_data(self):
        print(self.data * 100)


@jitclass
class Bar:
    logger: UnionType[Foo, Boo]

    def __init__(self, foo_data=None, boo_data=None):
        if foo_data:
            self.logger = Foo(foo_data)
        elif boo_data:
            self.logger = Boo(boo_data)

ImportError: cannot import name 'UnionType' from 'numba' (e:\Coding\venvs\qfree\lib\site-packages\numba\__init__.py)

In [41]:
bar.logger.multi_data(x)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mtoo many positional arguments[0m
[0m[1m- Resolution failure for non-literal arguments:
[1mNone[0m
[0m[0m
[0m[1mDuring: resolving callee type: BoundFunction((<class 'numba.core.types.misc.ClassInstanceType'>, 'multi_data') for instance.jitclass.Foo#1a22f37f250<data:array(int64, 1d, A)>)[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[1m
File "<string>", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m
